In [1]:
# Imports
import os
import gradio as gr
from typing import cast
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from pydantic import BaseModel

In [ ]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [ ]:
# Load Environment Variables
load_dotenv(override=True)